# Common Table Expression (CTE)
CTEs ถือเป็นนิพจน์ตาราง (Table Expression) แบบหนึ่ง
นิพจน์ตาราง คือ นิพจน์ที่ทำหน้าที่เสมือนตารางได้
 
มันคือการเขียนคำสั่ง SQL query ให้ทำงานแบบต่อเนื่อง (จะเป็น recursive หรือตามลำดับชั้น hierarchy ก็ได้)

In [23]:
select * from sales order by random() limit 20;

(20 row(s) affected)

Total execution time: 00:00:00.010

branch,dt,revenue
phuket,2018-02-03,776
phuket,2018-12-27,487
phuket,2019-12-11,511
phuket,2018-04-02,686
phuket,2019-06-21,514
bangkok,2019-09-15,513
phuket,2019-10-27,154
phuket,2019-04-06,430
bangkok,2018-11-22,739
phuket,2018-01-02,569


In [24]:
select branch, date_part('month', dt)::int "month", sum(revenue) total
    from sales
    where date_part('year', dt) = 2018
    group by branch, date_part('month', dt) 
    order by branch, "month";

(24 row(s) affected)

Total execution time: 00:00:00.004

branch,month,total
bangkok,1,20728
bangkok,2,15822
bangkok,3,17184
bangkok,4,21011
bangkok,5,18634
bangkok,6,14029
bangkok,7,17666
bangkok,8,16647
bangkok,9,17154
bangkok,10,19355


In [25]:
with cte as(
    select branch, date_part('month', dt)::int "month", sum(revenue) total
    from sales
    where date_part('year', dt) = 2018
    group by branch, date_part('month', dt) 
    order by branch, "month" 
)

select * from cte

(24 row(s) affected)

Total execution time: 00:00:00.003

branch,month,total
bangkok,1,20728
bangkok,2,15822
bangkok,3,17184
bangkok,4,21011
bangkok,5,18634
bangkok,6,14029
bangkok,7,17666
bangkok,8,16647
bangkok,9,17154
bangkok,10,19355


In [26]:
with cte as(
    select branch, date_part('month', dt)::int "month", sum(revenue) total
    from sales
    where date_part('year', dt) = 2018
    group by branch, date_part('month', dt) 
    order by branch, "month" 
)

select b.month, b.total "bangkok", p.total "phuket", b.total - p.total "Bkk-Phuket"
    from cte b inner join cte p using(month)
    where b.branch = 'bangkok' and p.branch = 'phuket'

(12 row(s) affected)

Total execution time: 00:00:00.013

month,bangkok,phuket,Bkk-Phuket
1,20728,14800,5928
2,15822,13807,2015
3,17184,13293,3891
4,21011,12962,8049
5,18634,14311,4323
6,14029,12837,1192
7,17666,12723,4943
8,16647,14400,2247
9,17154,13060,4094
10,19355,14421,4934


In [27]:
with cte as(
    select branch, date_part('month', dt)::int "month", sum(revenue) total
    from sales
    where date_part('year', dt) = 2018
    group by branch, date_part('month', dt) 
    order by branch, "month" 
),
b as(
    select * from cte where branch = 'bangkok'
), 
p as(
    select * from cte where branch = 'phuket'
)


select b.month, b.total "bangkok", p.total "phuket", b.total - p.total "Bkk-Phuket",
    to_char(b.total - p.total, '99G999G999')
    from b inner join p using(month)
   

(12 row(s) affected)

Total execution time: 00:00:00.004

month,bangkok,phuket,Bkk-Phuket,to_char
1,20728,14800,5928,"5,928"
2,15822,13807,2015,"2,015"
3,17184,13293,3891,"3,891"
4,21011,12962,8049,"8,049"
5,18634,14311,4323,"4,323"
6,14029,12837,1192,"1,192"
7,17666,12723,4943,"4,943"
8,16647,14400,2247,"2,247"
9,17154,13060,4094,"4,094"
10,19355,14421,4934,"4,934"


In [28]:
 select branch, date_part('year', dt)::int "year",
    date_part('month', dt)::int "month", sum(revenue) total
    from sales
    -- where date_part('year', dt) = 2018
    group by branch, date_part('year', dt), date_part('month', dt) 
    order by branch, "year", "month" 

(48 row(s) affected)

Total execution time: 00:00:00.005

branch,year,month,total
bangkok,2018,1,20728
bangkok,2018,2,15822
bangkok,2018,3,17184
bangkok,2018,4,21011
bangkok,2018,5,18634
bangkok,2018,6,14029
bangkok,2018,7,17666
bangkok,2018,8,16647
bangkok,2018,9,17154
bangkok,2018,10,19355


In [33]:
with cte as(
    select branch, date_part('year', dt)::int "year",
    date_part('month', dt)::int "month", sum(revenue) total
    from sales
    -- where date_part('year', dt) = 2018
    group by branch, date_part('year', dt), date_part('month', dt) 
    order by branch, "year", "month" 
),
b as(
    select * from cte where branch = 'bangkok'
), 
p as(
    select * from cte where branch = 'phuket'
)

select b.year, b.month, b.total "bangkok", p.total "phuket", b.total - p.total "Bkk-Phuket",
    to_char(b.total - p.total, '99G999G999')
    from b inner join p on b.month = p.month

(48 row(s) affected)

Total execution time: 00:00:00.004

year,month,bangkok,phuket,Bkk-Phuket,to_char
2018,1,20728,14800,5928,"5,928"
2018,1,20728,14264,6464,"6,464"
2018,2,15822,13807,2015,"2,015"
2018,2,15822,12450,3372,"3,372"
2018,3,17184,13293,3891,"3,891"
2018,3,17184,12731,4453,"4,453"
2018,4,21011,12962,8049,"8,049"
2018,4,21011,14358,6653,"6,653"
2018,5,18634,14311,4323,"4,323"
2018,5,18634,11660,6974,"6,974"


In [36]:
with cte as(
    select branch, date_part('year', dt)::int "year",
    date_part('month', dt)::int "month", sum(revenue) total
    from sales
    -- where date_part('year', dt) = 2018
    group by branch, date_part('year', dt), date_part('month', dt) 
    order by branch, "year", "month" 
),
b as(
    select * from cte where branch = 'bangkok'
), 
p as(
    select * from cte where branch = 'phuket'
)

select b.year, b.month, b.total "bangkok", p.total "phuket", b.total - p.total "Bkk-Phuket",
    to_char(b.total - p.total, '99G999G999')
    from b inner join p on b.year = p.year and b.month = p.month

(24 row(s) affected)

Total execution time: 00:00:00.007

year,month,bangkok,phuket,Bkk-Phuket,to_char
2018,1,20728,14800,5928,"5,928"
2018,2,15822,13807,2015,"2,015"
2018,3,17184,13293,3891,"3,891"
2018,4,21011,12962,8049,"8,049"
2018,5,18634,14311,4323,"4,323"
2018,6,14029,12837,1192,"1,192"
2018,7,17666,12723,4943,"4,943"
2018,8,16647,14400,2247,"2,247"
2018,9,17154,13060,4094,"4,094"
2018,10,19355,14421,4934,"4,934"
